In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 

data=[]

def get_article_content(article_url,mer):
  r = requests.get(article_url)
  soup = BeautifulSoup(r.text, "lxml")
  results = soup.select('span.article-meta-value')
  if results:
    if (results[2].text.find("販售")!=-1)&(results[2].text.find(mer)!=-1)&(results[2].text.find("已售出")==-1): 
        #取出文章中的價格
        url = article_url
        response = requests.get(url)
        
        #將原始碼做整理
        soup = BeautifulSoup(response.text, 'lxml')
        date = checkformat(soup, '.article-meta-value', 'date', 3, url)
        
        #content 文章內文
        content = soup.find(id="main-content").text
        target_content = u'※ 發信站: 批踢踢實業坊(ptt.cc),'
        #去除掉 target_content
        content = content.split(target_content)
        #print(content)
        content = content[0].split(date)
        #print(content)
        #去除掉文末 --
        main_content = content[1].replace('--', '')  
        
        
        if(main_content.find('售出') == -1):

            pr_1 = main_content.find('[售價]')

            flag=False
            price = ""
            
            if(pr_1 != -1):
         
                for i in range(100):
                    if((main_content[pr_1+4+i])==',' 
                       or (main_content[pr_1+4+i])=='$'
                       or (main_content[pr_1+4+i])==' '
                       or (main_content[pr_1+4+i])=='N'
                       or (main_content[pr_1+4+i])=='T'
                       ):
                        continue
                    elif((main_content[pr_1+4+i])=='\n')&(i<3):
                        continue
                    elif((main_content[pr_1+4+i])=='\n'):
                        break
                    elif(48>ord(main_content[pr_1+4+i]) or ord(main_content[pr_1+4+i])>57):
                        flag=True
                        price += main_content[pr_1+4+i]
                    else:
                        price += main_content[pr_1+4+i]
            #print(price)
            if(flag==True or price==""):
                print('作者:', results[0].text)
                #print('看板:', results[1].text)
                print('標題:', results[2].text)
                print('時間:', results[3].text)
                print('價格:', price) 
                print("IRL:", article_url)
                print()
            else:
                price=int(price)
                data.append({
                    '作者':results[0].text,
                    '標題':results[2].text,
                    '時間':results[3].text,
                    '價格':price,
                    "IRL":article_url
                    })


def checkformat(soup, class_tag, data, index, url):
   content = soup.select(class_tag)[index].text
   return content
        
def get_all_href(url,mer):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    results = soup.select("div.title")
    for item in results:
        a_item = item.select_one("a")
        if a_item:
            article_url='https://www.ptt.cc'+a_item.get('href')
            get_article_content(article_url,mer)

print("請輸入要尋找哪項商品?")
print("1.iPad")
print("2.iPhone")
print("3.AirPods")
merch = int(input())
print()

if(merch==1):
    mer = "iPad"
elif(merch==2):
    mer = "iPhone"
elif(merch==3):
    mer = "AirPods"

url="https://www.ptt.cc/bbs/MacShop/index.html"

for page in range(1,10):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    btn = soup.select('div.btn-group > a')
    
    if btn:
        next_page_url = 'https://www.ptt.cc' + btn[3]['href']
        url = next_page_url
        get_all_href(url,mer)

df = pd.DataFrame(data)
df = df.sort_values(by=['價格'])
pd.set_option('display.max_columns',1000)
display(df)

請輸入要尋找哪項商品?
1.iPad
2.iPhone
3.AirPods
1



,作者,標題,時間,價格,IRL
4,JAMES10033 (小周),[販售] 高雄 iPad 9 64G 灰色,Tue Jan 23 23:32:49 2024,7500,https://www.ptt.cc/bbs/MacShop/M.1706023971.A....
0,lightmyfire (...),[販售] 新竹 二手iPad 9 (Wifi) 64GB,Thu Jan 25 09:38:45 2024,8500,https://www.ptt.cc/bbs/MacShop/M.1706146727.A....
3,sentana (鄉民),[販售] 新竹 iPad 9th WIFI 64GB 銀 全新,Wed Jan 24 10:02:32 2024,9000,https://www.ptt.cc/bbs/MacShop/M.1706061754.A....
5,jimmymmy1 (jimmymmy1),[販售] 新竹 iPad 9th WIFI 64GB 灰,Wed Jan 24 01:54:56 2024,9000,https://www.ptt.cc/bbs/MacShop/M.1706032498.A....
2,supertty (小聖),[販售] 台中 iPad 9 64GB wifi 太空灰,Wed Jan 24 08:52:03 2024,9200,https://www.ptt.cc/bbs/MacShop/M.1706057525.A....
1,cyclonejoker (Φ),[販售] 全國 iPad Air 5 64G 太空灰 已加保appl,Wed Jan 24 13:03:02 2024,17500,https://www.ptt.cc/bbs/MacShop/M.1706072584.A....
